In [1]:
import pymysql
import pandas as pd
import re
from bs4 import BeautifulSoup
from hanziconv import HanziConv
from zhon import hanzi
import string
import html
import pymysql
from tqdm import tqdm
import json

def clear_text(x):
    x = BeautifulSoup(x, 'html.parser').text
    x = html.unescape(x)
    x = HanziConv.toSimplified(x)
    x = re.sub(r'\s+', '', x) # tab \n 去除 匹配任何非空白字符。等价于 [^ \f\n\r\t\v]。
    x = re.sub(r'[\(（【](.*?)[\)）】]', '', x) # 替换 （）是单元， ?非贪婪匹配，否则.*把后面的括号也匹配掉了
    x = re.sub(r'([–-—=…]*)', '',x)
    x = x.strip()
    return x

def from_sql(sql):
    conn = pymysql.connect(host='127.0.0.1',port=3306,user='root',password='eXYhzAWjyvy8grwM',charset='utf8mb4')
    df = pd.read_sql(sql, conn)  
    conn.close()
    return df

In [2]:
# 王者荣耀 2301
# 绝地求生*刺激战场 70056
# 香肠派对 58881
# 碧蓝航线 31597
game_id = 31597
sql = """
SELECT source,game_name, game_id, score, up, content FROM game_source.s_game_comments_taptap_game where game_id = %s  and cast(up as signed)>5 
"""% (game_id,)
df = from_sql(sql)

/home/mingchao/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:165: Warning: (1292, "Truncated incorrect INTEGER value: ''")
  result = self._query(query)


In [3]:
# 去除\n，替换空格 和（xxx） 
df['content_clear'] = df['content'].apply(clear_text)

In [4]:
df.shape

(1578, 7)

In [5]:
import nltk
import jieba
jieba.load_userdict('comment_jieba_add_word.txt')
import jieba.posseg as pseg
def leaves(tree):
    for subtree in tree.subtrees(filter=lambda t: t.label()=='NP'):
        yield subtree.leaves()
def extract_asp_by_grammar(text):     
    # text = "因为这个风格蛮喜欢的,加上TapTap的推荐就下载来玩了,bgm挺好的游戏（开着bgm写的评论）,然后游戏体验相当棒了"
    words = pseg.cut(text)
    sent_tags = [(x,y) for x,y in words]
    # NP(名词短语块) DT(限定词) JJ(形容词) NN(名词)
    # 名词名词，或者名词形容词
    # grammar = "NP: {<n>{2,4}}"
    grammar = r"""
                NP: {<n><n><d>*<a>+}
                    {<v><n><d>*<a>+}
                """
    # 进行分块
    cp = nltk.RegexpParser(grammar)
    tree = cp.parse(sent_tags)
    aspects = [[w for w,t in leaf] for leaf in leaves(tree)]
    return [''.join(asp) for asp in aspects]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.627 seconds.
Prefix dict has been built succesfully.


In [6]:
def sents_len(content_clear):
    sents = re.split(r'[!。！? ？]', content_clear)
    sents = [x for x in sents if x is not '']
    return len(sents)

In [7]:
# extract_asp_by_grammar('贪恋着钻石')

In [8]:
import jieba.posseg as pseg
def extract_sents_aspects(content_clear):
    sents = re.split(r'[!。！? ？]', content_clear)
    sents = [x for x in sents if x is not '']
    sents_tags = [extract_asp_by_grammar(text) for text in sents] # [[],[],[]]
    #是否要去掉，单字特征
    return [y for x in sents_tags if x!=[] for y in x if len(y)>1] # [ ]

In [9]:
# sentens_aspects
df['sents_asp'] = df['content_clear'].apply(extract_sents_aspects)

In [10]:
df['sents_len'] = df['content_clear'].apply(sents_len)

In [11]:
df.head(1)

,source,game_name,game_id,score,up,content,content_clear,sents_asp,sents_len
0,taptap,碧蓝航线,31597,4.0,23,emmm。。。\n作为瓦尔基里区的头号优秀(搞事)段子手，玩过的RPG也只有造物法则和这个，...,emmm。。。作为瓦尔基里区的头号优秀段子手，玩过的RPG也只有造物法则和这个，没什么，休闲...,[],18


In [12]:
# 改善提取特征，
sents_aspects = [y for _,x in df.iterrows() for y in x['sents_asp']]

In [13]:
len(sents_aspects)

967

In [14]:
def get_n_v_a(s1):
    v_n = ''
    a = ''
    for x,y in pseg.cut(s1):
        if y=='v' or y=='n':
            v_n=v_n+x
        elif '不' in x or y=='a':
            a=a+x
        else:
            pass
    return v_n, a

In [15]:
asp_v_n = [(x,)+get_n_v_a(x) for x in sents_aspects]
asp_v_n_4 = [(x, y, z) for x,y,z in asp_v_n if len(y)>3]

In [16]:
asp_v_n_4

[('游戏爆率低', '游戏爆率', '低'),
 ('蓝装紫装同样很强', '蓝装紫装', '强'),
 ('潜艇定位尴尬', '潜艇定位', '尴尬'),
 ('出场机会好', '出场机会', '好'),
 ('游戏质量不错', '游戏质量', '不错'),
 ('再见分割线好', '再见分割线', '好'),
 ('建造全部固定', '建造全部', '固定'),
 ('开心船厂全', '开心船厂', '全'),
 ('数值平衡性很差', '数值平衡性', '差'),
 ('说平衡性很差', '说平衡性', '差'),
 ('妹子态度挺好', '妹子态度', '好'),
 ('是活不长', '是活不长', ''),
 ('游戏寿命不长', '游戏寿命', '不长'),
 ('意味着成本越高', '意味着成本', '高'),
 ('新手教程不足', '新手教程', '不足'),
 ('资源收支严重', '资源收支', '严重'),
 ('显得资源不足', '显得资源', '不足'),
 ('游戏手感却很差', '游戏手感', '差'),
 ('战列打击面太窄', '战列打击面', '窄'),
 ('成分比重很大', '成分比重', '很大'),
 ('是认同度相当高', '是认同度', '高'),
 ('历史考据不严谨', '历史考据', '不严谨'),
 ('游戏资历较老', '游戏资历', '老'),
 ('认为舰娘就是简单', '认为舰娘', '简单'),
 ('希望官方良心', '希望官方', '良心'),
 ('更换装备就好', '更换装备', '好'),
 ('蓝皮充数都好', '蓝皮充数', '好'),
 ('资源收支严重失衡', '资源收支', '严重失衡'),
 ('模型钻石很难', '模型钻石', '难'),
 ('导致时间紧张', '导致时间', '紧张'),
 ('人类游戏不同', '人类游戏', '不同'),
 ('让消费力相对较低', '让消费力', '低'),
 ('舰类游戏一般', '舰类游戏', '一般'),
 ('传统手游非常常见', '传统手游', '常见'),
 ('游戏质量好', '游戏质量', '好'),
 ('看来官方根本', '看来官方', '根本'),
 ('氪金投入产出比较好', '氪金投入产出', '好'),


In [17]:
### 相似融合


In [18]:
from collections import Counter
sentences_len = sum(df['sents_len'])
# count/sentences_len > 0.00017
asp_n_n = [y for x,y,z in asp_v_n_4]
aspects = [(asp,count) for asp,count in Counter(asp_n_n).most_common(10) if count/sentences_len > 0.00017]

In [19]:
# a = set([z for x in aspects for _,y,z in asp_v_n_4  if x in y])

In [20]:
# 查看 应用宝的评论文本的 模型
# from gensim.models import KeyedVectors
# qq_vector = KeyedVectors.load("taptap_comments_vector", mmap='r')

In [21]:
# qq_vector.n_similarity(['希望', '官方'], [ '游戏','体验'])

In [22]:
# write 
def write_txt(text, t_name):
    with open(t_name, 'a') as f:
        for x in text:
            f.write(x+'\n')

In [23]:
sentiment_dict = {}
with open('lcs1.txt', 'r') as f:
    for x in f.readlines():
        try:
            x = x.strip()
            sentiment_dict[x.split(' ')[0]] = x.split(' ')[1]
        except:
            print(x.strip().split(' ')[0],'dict key is exist')

In [24]:
sentiment_dict

{'不低': '1',
 '不满': '0',
 '不错': '1',
 '不高': '0',
 '丰富': '1',
 '低': '0',
 '全': '1',
 '复杂': '0',
 '好': '1',
 '好低': '0',
 '少': '0',
 '巨爽': '1',
 '差': '0',
 '很棒': '1',
 '快': '1',
 '挺好玩': '1',
 '流畅': '1',
 '良心': '1',
 '诚恳': '1',
 '过硬': '1',
 '远': '0',
 '适当平衡': '1',
 '高': '1'}

In [26]:
### 分开成
asp_n = [] # n代表消极
asp_p = [] # p代表积极
for asp,_ in aspects:
    for x,y,z in asp_v_n_4:
        if asp in y:
            try:
                if sentiment_dict[z] == '1':
                    asp_p.append((x,y,z))
                elif sentiment_dict[z] == '0':
                    asp_n.append((x,y,z))
            except:
                write_txt([z], 'comment_sentiment_miss_word.txt')

In [27]:
asp_n

[('游戏体验差', '游戏体验', '差'),
 ('游戏体验很差', '游戏体验', '差'),
 ('游戏体验差', '游戏体验', '差'),
 ('游戏质量远', '游戏质量', '远'),
 ('氪金体验差', '氪金体验', '差'),
 ('金皮爆率也太低', '金皮爆率', '低'),
 ('金皮爆率还低', '金皮爆率', '低'),
 ('游戏内容太少', '游戏内容', '少'),
 ('游戏内容不满', '游戏内容', '不满'),
 ('游戏内容复杂', '游戏内容', '复杂'),
 ('游戏评分不高', '游戏评分', '不高'),
 ('游戏评分好低', '游戏评分', '好低'),
 ('游戏评分低', '游戏评分', '低'),
 ('觉得游戏性不高', '觉得游戏性', '不高')]

In [28]:
asp_p

[('游戏体验还不错', '游戏体验', '不错'),
 ('游戏体验流畅', '游戏体验', '流畅'),
 ('游戏体验好', '游戏体验', '好'),
 ('游戏体验很好', '游戏体验', '好'),
 ('游戏体验巨爽', '游戏体验', '巨爽'),
 ('游戏体验不错', '游戏体验', '不错'),
 ('游戏体验真的不错', '游戏体验', '不错'),
 ('游戏体验很好', '游戏体验', '好'),
 ('游戏体验不错', '游戏体验', '不错'),
 ('游戏体验不错', '游戏体验', '不错'),
 ('游戏体验很棒', '游戏体验', '很棒'),
 ('游戏体验非常好', '游戏体验', '好'),
 ('游戏体验不错', '游戏体验', '不错'),
 ('官方态度诚恳', '官方态度', '诚恳'),
 ('官方态度诚恳', '官方态度', '诚恳'),
 ('官方态度很好', '官方态度', '好'),
 ('官方态度还不错', '官方态度', '不错'),
 ('官方态度不错', '官方态度', '不错'),
 ('官方态度好', '官方态度', '好'),
 ('官方态度很好', '官方态度', '好'),
 ('官方态度也挺好', '官方态度', '好'),
 ('官方态度很好', '官方态度', '好'),
 ('游戏质量不错', '游戏质量', '不错'),
 ('游戏质量好', '游戏质量', '好'),
 ('游戏质量不错', '游戏质量', '不错'),
 ('游戏质量过硬', '游戏质量', '过硬'),
 ('游戏质量高', '游戏质量', '高'),
 ('游戏质量还不错', '游戏质量', '不错'),
 ('氪金体验也还不错', '氪金体验', '不错'),
 ('氪金体验就很好', '氪金体验', '好'),
 ('氪金体验不错', '氪金体验', '不错'),
 ('氪金体验很好', '氪金体验', '好'),
 ('金皮爆率高', '金皮爆率', '高'),
 ('金皮爆率高', '金皮爆率', '高'),
 ('金皮爆率很高', '金皮爆率', '高'),
 ('金皮爆率高', '金皮爆率', '高'),
 ('金皮爆率也不错', '金皮爆率', '不错'),
 ('金皮爆率不低', '金

In [29]:
# 统计
def asp_content_count(asp_p):
    asp_p_n = [y for _,y,_ in asp_p]
    asp_p_n_count = [(asp,count) for asp,count in Counter(asp_n_n).most_common(20)]
    # 找出区别
    asp_p_content = [x for x,_,_ in asp_p]
    asp_p_content_count = []
    for x,y in asp_p_n_count:
        for content in asp_p_content:
            if x in content:
                a = content + '(' + str(y) + ')'
                asp_p_content_count.append(a)
                break
    return asp_p_content_count

In [34]:
p = asp_content_count(asp_p)

In [35]:
n = asp_content_count(asp_n)

In [33]:
for x,y in pseg.cut('觉得游戏已经全'):
    print(x,y)

觉得 v
游戏 n
已经 d
全 a


In [37]:
dict(zip(['p', 'n'],[p,n]))

{'n': ['游戏体验差(16)',
  '游戏质量远(8)',
  '氪金体验差(8)',
  '金皮爆率也太低(8)',
  '游戏内容太少(4)',
  '游戏评分不高(3)',
  '觉得游戏性不高(3)'],
 'p': ['游戏体验还不错(16)',
  '官方态度诚恳(10)',
  '游戏质量不错(8)',
  '氪金体验也还不错(8)',
  '金皮爆率高(8)',
  '希望游戏越来越好(5)',
  '游戏内容丰富(4)',
  '希望官方良心(3)',
  '游戏素质很好(3)',
  '游戏模式相当不错(3)',
  '觉得游戏已经全(3)']}